# Import all required Modules and custom build Classes into the Jupyter notebook

In [1]:
import sys
import os
import json
import time
import pymongo
import urllib
import pandas as pd
import numpy as np
import urllib
import urllib.request

try:
    import createNewData.data.config as in_config
    from createNewData.pypackages.Azure import Azure
    from createNewData.pypackages.urlHandler import UrlHandler
    AzurePackage = Azure(in_config)
    Url = UrlHandler(in_config)
    
    
except ImportError as e:
    print("Failed to import critical modules for this script.")
    print("Please confirm that files exist in the correct locations.")
    print(e)

# Upload to SQL Database
The below can upload an entire table from a txt or csv file to the Azure SQL Database.
Required fields:
* "UploadToSQL" is the class name for uploading tables to the database.
* tablepath is the path the file is located in.
* tablename is the name of the new table to be created.

In [2]:
df = pd.read_csv(in_config.shapes)

SqlDataCursor = AzurePackage("UploadToSQL",
                              df,
                              "shapes")

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\James\\Documents\\MSc in Data Analytics\\Database and Ananytics\\Research Project\\dapTbElectricDublinBus\\ingestRawData\\raw\\shapes.txt'

# Return All Unique Shape Id's From SQL Database
The below collects all unique values in a specific column of a database.
Required fields:
* "SelectDistinct" is the class name for collecting the unique values.
* shape_id is the column name.
* [[dbo].[shapes]] is the name of the table to be queried.

In [3]:
shapeIds = AzurePackage("SelectDistinct",
                              "shape_id",
                              "[dbo].[shapes]")

# Collect Elevation Data and Upload to MongoDB
The below collects the longtitude and lattitude for elevation data from the shapes table.
It then generates and send the request to the open elevations URL.
Finally it stores this data to the mongoDB database in Azure Cosmos.

In [14]:
shapeList = []
AzurePackage("DropMongoColl","shapes")
for each in shapeIds.iterrows():
    try:
        # Generate the Pandas table of all the Longtitudes and Latitudes
        # for each shape
        elevations = AzurePackage("SelectLongLat",
                                "[shape_id],[shape_pt_lat],[shape_pt_lon]",
                                "[dbo].[shapes]",
                                "[shape_id]",
                                each[1][0])

        # Generate the Json document for upload to MongoDB
        shapeData = Url("mineElevationData",elevations)
    except urllib.error.HTTPError as e:
        print(in_config.URLOOD)
        print(e)

    try:
        # Upload the Json document to MongoDB
        if not len(shapeData) == 0:
            AzurePackage("UploadToMongo","shapes",shapeData)
        else:
            raise Exception(in_config.NDIDF)
    except TypeError as e:
        print(in_config.TEC)
    except pymongo.errors.DuplicateKeyError as e:
        print(in_config.FIDB)
    except Exception as e:
        print(in_config.UNKMGO)
        print(e)



# Read Data From MongoDB and Write to Pandas DataFrame
This reads data from the raw Json files in the MongoDB database and imports them to a dataframe that contains only unique values. 
Removing any rows where any full row duplicates exist.<br />
Finally, this tests that all the elevation data was collected correctly by summing the values of the elevations and therefore ruling out any NaN values. An exception will be raised here in the final script to indicate that the elevation collection was unsuccessfull.

In [15]:
listOfObjects = []
listOfElevations = []


try:
    dbcollections = AzurePackage("SelectFromMongo")
    for each in dbcollections.find():
        for key, value in each.items():
            if type(value) is list:
                listOfObjects.append(value)
        

    for each in listOfObjects:
        for elevation in each:
            listOfElevations.append(elevation)

    df = pd.DataFrame(listOfElevations)
    dfTrimmed = df.drop_duplicates()
    sumElevation = dfTrimmed["elevation"].sum()
    if type(sumElevation) in [np.int64,int]:
        print("Elevations collected correctly")
        print(dfTrimmed.head())
    else: 
        raise Exception("Failed to collect all elevations, please try again.")
except KeyError as e:
    print(f"Column {e} cannot be found in the dataframe.")
except NameError as e:
    print(f"The Datatable {e} cannot be found.")
except Exception as e:
    print(in_config.UNKMGO)
    print(e)

Elevations collected correctly
    latitude  elevation  longitude
0  53.391176         55  -6.262199
1  53.391189         55  -6.262439
2  53.391871         59  -6.262366
3  53.391854         59  -6.260548
4  53.391815         58  -6.259722


# Upload Trimmed Elevations DataFrame to SQL

This saves the resulting SQL schemata to the development database.
This will overwrite any existing data in the SQL schemata that already exists.


In [7]:
SqlDataCursor = AzurePackage("UploadToSQL",
                              dfTrimmed,
                              "elevations",
                              in_config.connQuote)

# Collect real time data and upload to Mongo
This collects the real time data as a json file and overwrites the collection in the mongoDB database.
* URL used https://gtfsr.transportforireland.ie


In [11]:
url = in_config.url2
headers = in_config.RTIheaders
response = Url("callURL", url, {}, headers)
JsonData = response.read().decode('utf8').replace("'", '"')
RTIgtfs = json.loads(JsonData)
try:
    AzurePackage("DropMongoColl","RTIgtfs")
    AzurePackage("UploadToMongo","RTIgtfs",RTIgtfs)
except pymongo.errors.WriteError as e:
    print("An error occured while attempting to write the GTFS data to Mongo DB.")
    print(type(e))


An error occured while attempting to write the GTFS data to Mongo DB.
<class 'pymongo.errors.WriteError'>


# Save Trimmed Elevation Data Team SQL Database
This reads data from the raw Json files in the MongoDB database and imports them to a dataframe that contains only unique values. Removing any rows where any full row duplicates exist.
This saves the resulting SQL schemata to the Database used by the R-Shiny app. (Production)
This will overwrite any existing data in the SQL schemata that already exists.

In [17]:
SqlDataCursor = AzurePackage("UploadToSQL",
                              dfTrimmed,
                              "elevations",
                              in_config.teamConnQuote)

# Connect the shape and elevation schema together by joining their longtitude and latatude values
This will either be a method in the Rshiny app to collect or we simply create a new database from this data but this seems a bit verbose.

In [ ]:
conn = AzurePackage("AzureDBConn", in_config.connQuote)
SQLString = in_config.SQLElevation
df = pd.read_sql(SQLString, conn)
conn.close()

In [23]:
type(dfTrimmed["elevation"].sum())

numpy.int64

In [3]:
conn = AzurePackage("AzureDBConn", in_config.teamConnQuote)
cursor = conn.cursor()
a = cursor.execute("SELECT COUNT(*) FROM [dbo].[elevations]")
print(a.fetchall())
conn.close()

[(19876, )]


In [4]:
shapeIds

,shape_id
0,60-1-b12-1.1.O
1,60-1-b12-1.2.O
2,60-1-b12-1.3.I
3,60-1-b12-1.4.I
4,60-1-d12-1.1.O
...,...
652,60-9-b12-1.13.I
653,60-9-d12-1.10.O
654,60-9-d12-1.11.O
655,60-9-d12-1.12.I
